In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math 
from tqdm import tqdm

from src.utils import print_yaml, rescale_logits, calculate_tau, plot_with_band, plot_bootstrap_band, calculate_roc, pick_weighted_models
from src.save_load import savePlot, loadFbdStudy, loadTargetSignals, loadShadowModelSignals, saveVisData, loadVisData
from LeakPro.leakpro.attacks.mia_attacks.lira import lira_vectorized
from LeakPro.leakpro.attacks.mia_attacks.rmia import rmia_vectorised, rmia_get_gtlprobs
from src.save_load import savePlot

In [ ]:
# -------------------------- #
#   Load fbd study results   #
# -------------------------- #
study_name = "cifar10-resnet-fbd-00c3fdac98"
study_path = os.path.join("study", study_name)
metadata = []
gtl_probs = []
resc_logits = []

metadata, fbd_trial_results, gtl_probs, resc_logits, labels = loadFbdStudy(study_name, metadata=True, gtl=True, logits=True)
target_folder = metadata["study"]["target_folder"]

In [ ]:
# -------------------------------- #
#   Load baseline target signals   #
# -------------------------------- #
target_folder = metadata["study"]["target_folder"]

baseline_logits, baseline_inmask, baseline_resc_logits, baseline_gtl_probs, baseline_metadata, baseline_metadata_pkl = loadTargetSignals(target_folder)

# Rescale baseline target logits
if baseline_resc_logits is None and baseline_logits is not None:
    baseline_resc_logits = rescale_logits(baseline_logits, labels)
print(f"Baseline resc_logits: {baseline_resc_logits[:10]}, shape: {baseline_resc_logits.shape}")

# Calculate the GTL Probabilities for the target logits
if baseline_gtl_probs is None and baseline_logits is not None:
    baseline_gtl_probs = rmia_get_gtlprobs(baseline_logits, labels)
print(f"Target gtl_probs: {baseline_gtl_probs[:10]}, shape: {baseline_gtl_probs.shape}")

baseline_accuracy = baseline_metadata_pkl.test_result.accuracy
print(f"Baseline accuracy: {baseline_accuracy}")

In [ ]:
# ----------------------------- #
#   Load shadow model signals   #
# ----------------------------- #
shadow_logits, rescaled_shadow_logits, shadow_gtl_probs, shadow_inmask, sm_metadata, missing_indices = loadShadowModelSignals(target_folder)

# Rescale and calc gtl for shadow models
# rescaled_shadow_logits and shadow_gtl_probs will be false if they are not loaded
if rescaled_shadow_logits is False or shadow_gtl_probs is False:
    N, M, C = shadow_logits.shape
    shadow_gtl_probs_list = []
    rescaled_shadow_logits_list = []
    for m in range(M):
        model_logits = shadow_logits[:, m, :]  # shape (N, C)
        
        if not shadow_gtl_probs:
            shadow_gtl_probs_list.append(rmia_get_gtlprobs(model_logits, labels))
        if not rescaled_shadow_logits:
            rescaled_shadow_logits_list.append(rescale_logits(model_logits, labels))
            
        print(f"{len(shadow_gtl_probs_list)} shadow gtl probs calculated and {len(rescaled_shadow_logits_list)} rescaled logits calculated")
    
    if not shadow_gtl_probs:    
        shadow_gtl_probs = np.stack(shadow_gtl_probs_list, axis=1)  # shape = (N, M)
    if not rescaled_shadow_logits:
        rescaled_shadow_logits = np.stack(rescaled_shadow_logits_list, axis=1)
    
print(f"shadow gtl probs shape: {shadow_gtl_probs.shape}, shadow resc logits shape: {rescaled_shadow_logits.shape}")

In [ ]:
# ----------------------------- #
#   Audit the baseline target   #
# ----------------------------- #
# Audit the baseline target with LiRA
lira_scores = lira_vectorized(baseline_resc_logits, rescaled_shadow_logits, shadow_inmask, "carlini", online=True)
print(f"lira_scores first 10: {lira_scores[:10]}")
# Audit the baseline target with RMIA
rmia_scores = rmia_vectorised(baseline_gtl_probs, shadow_gtl_probs, shadow_inmask, online=True, use_gpu_if_available=True)
print(f"rmia_scores first 10: {rmia_scores[:10]}")

# ------------------------------------- #
#   Audit or Load the weighted target   #
# ------------------------------------- #
data = loadVisData(study_name)

# Audit all weighted models with RMIA
if data.get('w_rmia_scores') is None:
    weighted_rmia_scores = []
    for gtl in tqdm(gtl_probs, desc=f"gtl_probs"):
        weighted_rmia_scores.append(rmia_vectorised(gtl, shadow_gtl_probs, shadow_inmask, online=True, use_gpu_if_available=True))
    saveVisData(np.stack(weighted_rmia_scores, axis=0), "w_rmia_scores", study_name)
else:
    weighted_rmia_scores = data.get('w_rmia_scores')
# Audit all weighted models with RMIA
if data.get('w_lira_scores') is None:
    weighted_lira_scores = []
    for resc_logits in tqdm(resc_logits, desc=f"resc_logits"):
        weighted_lira_scores.append(lira_vectorized(resc_logits, rescaled_shadow_logits, shadow_inmask, "carlini", online=True))
    saveVisData(np.stack(weighted_lira_scores, axis=0), "w_lira_scores", study_name)
else:
    weighted_lira_scores = data.get('w_lira_scores')
    
print(f"rmia_scores_weighted count: {len(weighted_rmia_scores)}")
print(f"lira_scores_weighted count: {len(weighted_lira_scores)}")

In [ ]:
# ----------------- #
#   Calculate tau   #
# ----------------- #
# Baseline
fpr1 = 0.1
fpr2 = 0.01
fpr3 = 0.001
tau_baseline_rmia_1 = calculate_tau(rmia_scores, baseline_inmask, fpr1)
tau_baseline_rmia_2 = calculate_tau(rmia_scores, baseline_inmask, fpr2)
tau_baseline_rmia_3 = calculate_tau(rmia_scores, baseline_inmask, fpr3)

tau_baseline_lira_1 = calculate_tau(lira_scores, baseline_inmask, fpr1)
tau_baseline_lira_2 = calculate_tau(lira_scores, baseline_inmask, fpr2)
tau_baseline_lira_3 = calculate_tau(lira_scores, baseline_inmask, fpr3)

print(f"baseline tau rmia_1: {tau_baseline_rmia_1} at fpr: {fpr1}")
print(f"baseline tau lira_1: {tau_baseline_lira_1} at fpr: {fpr1}")

# Weighted 0.1 fpr
weigted_taus_rmia_1 = []
weigted_taus_lira_1 = []
weigted_taus_rmia_2 = []
weigted_taus_lira_2 = []
weigted_taus_rmia_3 = []
weigted_taus_lira_3 = []

# --- RMIA ---
rmia_scores_count = 1
for w_rmia_score in tqdm(weighted_rmia_scores, desc=f"w_rmia_scores"):
    weigted_taus_rmia_1.append(calculate_tau(w_rmia_score, baseline_inmask, fpr1))
    weigted_taus_rmia_2.append(calculate_tau(w_rmia_score, baseline_inmask, fpr2))
    weigted_taus_rmia_3.append(calculate_tau(w_rmia_score, baseline_inmask, fpr3))
print(f"n rmia taus: {weigted_taus_rmia_1[:5]}, {weigted_taus_rmia_2[:5]}, {weigted_taus_rmia_3[:5]}")

# --- LIRA ---
lira_scores_count = 1
for w_lira_score in tqdm(weighted_lira_scores, desc=f"w_lira_scores"):
    weigted_taus_lira_1.append(calculate_tau(w_lira_score, baseline_inmask, fpr1))
    weigted_taus_lira_2.append(calculate_tau(w_lira_score, baseline_inmask, fpr2))
    weigted_taus_lira_3.append(calculate_tau(w_lira_score, baseline_inmask, fpr3))
print(f"n lira taus: {weigted_taus_lira_1[:5]}, {weigted_taus_lira_2[:5]}, {weigted_taus_lira_3[:5]}")

# Study outputs
accuracies = [res.accuracy for res in fbd_trial_results]
noises = [res.noise for res in fbd_trial_results]
centralities = [res.centrality for res in fbd_trial_results]
temperatures = [res.temperature for res in fbd_trial_results]
tau_rmia = [res.tau for res in fbd_trial_results]   # tau in this context is log(tauc_fbd@0.1/tauc_ref@0.1)
print(f"study tau: {tau_rmia[:5]}")

In [ ]:
# ----------------- #
#   Scatter plots   #
# ----------------- #

# Plot
fig, axes = plt.subplots(2, 2, figsize=(12, 5))
# --- Top-left: your study tau (FbD) --- #
ax = axes[0,0]
ax.scatter(tau_rmia, accuracies, label="FbD study", color="purple")
ax.set_xlabel("Study Objective: log(tauc_fbd@0.1 / tauc_ref@0.1)")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Top-right: tau@0.1 for RMIA + LIRA --- #
ax = axes[0,1]
ax.scatter(weigted_taus_rmia_1, accuracies, label="RMIA on FbD", color="cornflowerblue", marker='o')
ax.scatter(weigted_taus_lira_1, accuracies, label="LIRA on FbD", color="orange", marker='x')
ax.scatter(tau_baseline_rmia_1, baseline_accuracy, label="RMIA on Baseline", color="red", marker='o')
ax.scatter(tau_baseline_lira_1, baseline_accuracy, label="LIRA on Baseline", color="green", marker='x')
ax.set_xlabel("τ@0.1FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Bottom-left: tau@0.01 --- #
ax = axes[1,0]
ax.scatter(weigted_taus_rmia_2, accuracies, label="RMIA on FbD", color="cornflowerblue", marker='o')
ax.scatter(weigted_taus_lira_2, accuracies, label="LIRA on FbD", color="orange", marker='x')
ax.scatter(tau_baseline_rmia_2, baseline_accuracy, label="RMIA on Baseline", color="red", marker='o')
ax.scatter(tau_baseline_lira_2, baseline_accuracy, label="LIRA on Baseline", color="green", marker='x')
ax.set_xlabel("τ@0.01FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Bottom-right: tau@0.001 --- #
ax = axes[1,1]
ax.scatter(weigted_taus_rmia_3, accuracies, label="RMIA on FbD", color="cornflowerblue", marker='o')
ax.scatter(weigted_taus_lira_3, accuracies, label="LIRA on FbD", color="orange", marker='x')
ax.scatter(tau_baseline_rmia_3, baseline_accuracy, label="RMIA on Baseline", color="red", marker='o')
ax.scatter(tau_baseline_lira_3, baseline_accuracy, label="LIRA on Baseline", color="green", marker='x')
ax.set_xlabel("τ@0.001FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

savePlot(fig, "Scatter_plots", study_name, "study")

In [ ]:
# ----------------------- #
#  Calculate ROC Metrics  #
# ----------------------- #
baseline_tpr_curve_lira, baseline_fpr_curve_lira = calculate_roc(lira_scores, baseline_inmask, clip=True)
baseline_tpr_curve_rmia, baseline_fpr_curve_rmia = calculate_roc(rmia_scores, baseline_inmask, clip=True)

# Thresholds will be base accuracy -5% in steps
w_model_acc_threshold = [baseline_accuracy-0.05, baseline_accuracy-0.10, baseline_accuracy-0.15]

# Find indices that lie within a +2 range of a threshold index. Return the indices for each threshold.
# For each index list compare all tau
print(f"Selecting model indices by best vulnerability mitigate at FPR={fpr1}")
_, best_model_indices = pick_weighted_models(accuracies, weigted_taus_rmia_1, weigted_taus_lira_1, w_model_acc_threshold)


print(f"best rmia_indices: {best_model_indices}")
for idx in best_model_indices:
    print(f"accuracy: {accuracies[idx]}, tau@0.1: {weigted_taus_rmia_1[idx]}")
    print(f"accuracy: {accuracies[idx]}, tau@0.1: {weigted_taus_lira_1[idx]}")

w_rmia_tpr_curves = []
w_rmia_fpr_curves = []

w_lira_tpr_curves = []
w_lira_fpr_curves = []

w_acc = [accuracies[i] for i in best_model_indices]
w_noise = [noises[i] for i in best_model_indices]
w_cent = [centralities[i] for i in best_model_indices] 
w_temp = [temperatures[i] for i in best_model_indices] 
print(f"w_acc: {w_acc}, w_noise: {w_noise}, w_cent: {w_cent}, w_temp: {w_temp}")

for idx in best_model_indices:
    tpr_curve, fpr_curve = calculate_roc(weighted_rmia_scores[idx], baseline_inmask, clip=True)
    w_rmia_tpr_curves.append(tpr_curve)
    w_rmia_fpr_curves.append(fpr_curve)

    tpr_curve, fpr_curve = calculate_roc(weighted_lira_scores[idx], baseline_inmask, clip=True)
    w_lira_tpr_curves.append(tpr_curve)
    w_lira_fpr_curves.append(fpr_curve)

print(np.min(w_rmia_tpr_curves[2]), np.min(w_rmia_fpr_curves[2]))

In [ ]:
# ---------------------------------------------- #
#  ROC for baseline & different accuracy levels  #
# ---------------------------------------------- #
colors = ["orange", "green", "purple", "olive", "pink"][:len(best_model_indices)]

# ------- LiRA ROC ------- #
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ax = axes[0]
random_fpr = np.logspace(-5, 0, 500)   # from 1e-5 to 1 on log scale
random_tpr = random_fpr.copy()         # TPR = FPR
ax.plot(random_fpr, random_tpr, "--", color="red", alpha=0.7, label="Random Guessing")
ax.plot(baseline_fpr_curve_lira, baseline_tpr_curve_lira, color="cornflowerblue", label=f"Baseline: accuracy={baseline_accuracy:.2f}")
for tpr, fpr, color, acc, noise, cent, temp in zip(w_lira_tpr_curves, w_lira_fpr_curves, colors, w_acc, w_noise, w_cent, w_temp):
    ax.plot(fpr, tpr, color=color, label=f"FbD: accuracy={acc:.2f}, σ={noise:.2f}, c={cent:.2f} ,t={temp:.2f}")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(1e-5, 1)
ax.set_ylim(1e-5, 1)
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")
ax.grid(True, alpha=0.3)
ax.set_title("LiRA ROC Curves: Baseline vs. FbD Models at Target Accuracy Thresholds")
ax.legend(loc="lower right")

# ------- RMIA ROC ------- #
ax = axes[1]
random_fpr = np.logspace(-5, 0, 500)   # from 1e-4 to 1 on log scale
random_tpr = random_fpr.copy()         # TPR = FPR
ax.plot(random_fpr, random_tpr, "--", color="red", alpha=0.7, label="Random Guessing")
ax.plot(baseline_fpr_curve_rmia, baseline_tpr_curve_rmia, color="cornflowerblue", label=f"Baseline: accuracy={baseline_accuracy:.2f}")
for tpr, fpr, color, acc, noise, cent, temp in zip(w_rmia_tpr_curves, w_rmia_fpr_curves, colors, w_acc, w_noise, w_cent, w_temp):
    ax.plot(fpr, tpr, color=color, label=f"FbD: accuracy={acc:.2f}, σ={noise:.2f}, c={cent:.2f} ,t={temp:.2f}")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(1e-5, 1)
ax.set_ylim(1e-5, 1)
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")
ax.grid(True, alpha=0.3)
ax.set_title("RMIA ROC Curves: Baseline vs. FbD Models at Target Accuracy Thresholds")
ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

savePlot(fig, "roc_at_set_accuracy", study_name, "study")

In [ ]:
# ----------------------- #
#  STD Variability Bands  #
# ----------------------- #
fig, axes = plt.subplots(2, 2, figsize=(12, 5))

# --- Top-left: your study tau (FbD) --- #
ax = axes[0,0]
plot_with_band(ax, tau_rmia, accuracies, label="FbD study", color="purple")
ax.set_xlabel("Study Objective: log(tauc_fbd@0.1 / tauc_ref@0.1)")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Top-right: tau@0.1 for RMIA + LIRA --- #
ax = axes[0,1]
plot_with_band(ax, weigted_taus_rmia_1, accuracies, label="RMIA on FbD", color="cornflowerblue")
plot_with_band(ax, weigted_taus_lira_1, accuracies, label="LIRA on FbD", color="orange")
ax.set_xlabel("τ@0.1FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Bottom-left: tau@0.01 --- #
ax = axes[1,0]
plot_with_band(ax, weigted_taus_rmia_2, accuracies, label="RMIA on FbD", color="cornflowerblue")
plot_with_band(ax, weigted_taus_lira_2, accuracies, label="LIRA on FbD", color="orange")
ax.set_xlabel("τ@0.01FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Bottom-right: tau@0.001 --- #
ax = axes[1,1]
plot_with_band(ax, weigted_taus_rmia_3, accuracies, label="RMIA on FbD", color="cornflowerblue")
plot_with_band(ax, weigted_taus_lira_3, accuracies, label="LIRA on FbD", color="orange")
ax.set_xlabel("τ@0.001FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

savePlot(fig, "std_variability_bands", study_name, "study")

In [ ]:
# ----------------------------------- #
#  Bootstrapped Confidence Intervals  #
# ----------------------------------- #

fig, axes = plt.subplots(2, 2, figsize=(12, 5))

# --- Top-left: FbD tau --- #
ax = axes[0,0]
plot_bootstrap_band(ax, tau_rmia, accuracies, "FbD study", "purple")
ax.set_xlabel("Study Objective: log(tauc_fbd@0.1 / tauc_ref@0.1)")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Top-right: tau@0.1 for RMIA + LIRA --- #
ax = axes[0,1]
plot_bootstrap_band(ax, weigted_taus_rmia_1, accuracies, "RMIA on FbD", "cornflowerblue")
plot_bootstrap_band(ax, weigted_taus_lira_1, accuracies, "LIRA on FbD", "orange")
ax.set_xlabel("τ@0.1FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Bottom-left: tau@0.01 --- #
ax = axes[1,0]
plot_bootstrap_band(ax, weigted_taus_rmia_2, accuracies, "RMIA on FbD", "cornflowerblue")
plot_bootstrap_band(ax, weigted_taus_lira_2, accuracies, "LIRA on FbD", "orange")
ax.set_xlabel("τ@0.01FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

# --- Bottom-right: tau@0.001 --- #
ax = axes[1,1]
plot_bootstrap_band(ax, weigted_taus_rmia_3, accuracies, "RMIA on FbD", "cornflowerblue")
plot_bootstrap_band(ax, weigted_taus_lira_3, accuracies, "LIRA on FbD", "orange")
ax.set_xlabel("τ@0.001FPR")
ax.set_ylabel("Accuracy")
ax.grid(True, alpha=0.3)
ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

savePlot(fig, "confidence_intervals", study_name, "study")

In [ ]:
# ---------------------------------- #
#  Collect Groups & Calculate Pareto #
# ---------------------------------- #
grp_fpr = "0.01"    # Controls which group should be used

df = pd.DataFrame({
    "accuracy": accuracies,

    "study": np.arange(len(noises))+1,

    # RMIA / LiRA taus at different FPRs
    "tau_0.1_rmia": weigted_taus_rmia_1,
    "tau_0.1_lira": weigted_taus_lira_1,

    "tau_0.01_rmia": weigted_taus_rmia_2,
    "tau_0.01_lira": weigted_taus_lira_2,

    "tau_0.001_rmia": weigted_taus_rmia_3,
    "tau_0.001_lira": weigted_taus_lira_3,

    # Hyperparameters
    "noise": noises,
    "centrality": centralities,
    "temperature": temperatures,
})
print("========== df.head ==========")
df.head()
if grp_fpr == "0.001":
    group = (
        df.groupby(["noise", "centrality", "temperature"])
        .agg(
            accuracy_mean=("accuracy", "mean"),
            accuracy_std=("accuracy", "std"),

            rmia_mean=("tau_0.001_rmia", "mean"),
            rmia_std=("tau_0.001_rmia", "std"),

            lira_mean=("tau_0.001_lira", "mean"),
            lira_std=("tau_0.001_lira", "std"),
            
            n=("study", "size")
        )
        .reset_index()
    )
elif grp_fpr == "0.01":
    group = (
        df.groupby(["noise", "centrality", "temperature"])
        .agg(
            accuracy_mean=("accuracy", "mean"),
            accuracy_std=("accuracy", "std"),

            rmia_mean=("tau_0.01_rmia", "mean"),
            rmia_std=("tau_0.01_rmia", "std"),

            lira_mean=("tau_0.01_lira", "mean"),
            lira_std=("tau_0.01_lira", "std"),
            
            n=("study", "size")
        )
        .reset_index()
    )
elif grp_fpr == "0.1":
    group = (
        df.groupby(["noise", "centrality", "temperature"])
        .agg(
            accuracy_mean=("accuracy", "mean"),
            accuracy_std=("accuracy", "std"),

            rmia_mean=("tau_0.1_rmia", "mean"),
            rmia_std=("tau_0.1_rmia", "std"),

            lira_mean=("tau_0.1_lira", "mean"),
            lira_std=("tau_0.1_lira", "std"),
            
            n=("study", "size")
        )
        .reset_index()
    )
else:
    print(f"Invalid fpr: {grp_fpr}")
print("========== Group ==========")
group

tmp = group[group["n"]>=3]
xy_groups = []
xy_lira = tmp[["lira_mean", "accuracy_mean"]].to_numpy()
xy_groups.append(xy_lira)
xy_rmia = tmp[["rmia_mean", "accuracy_mean"]].to_numpy()
xy_groups.append(xy_rmia)

tst_list = []
for xy, attack in zip(xy_groups, ["lira", "rmia"]):
    n = xy.shape[0]
    is_dominated = np.zeros(n, dtype=bool)

    for i in range(n):
        # j dominates i if j has lower/equal x and higher/equal y,
        # and strictly better in at least one objective
        dominates_i = ((xy[:, 0] <= xy[i, 0]) & (xy[:, 1] >= xy[i, 1]) &
                    ((xy[:, 0] < xy[i, 0]) | (xy[:, 1] > xy[i, 1])))
        is_dominated[i] = np.any(dominates_i)

    frontier_mask = ~is_dominated
    if attack == "lira":
        tst = tmp.loc[frontier_mask].sort_values("lira_mean")
    elif attack == "rmia":
        tst = tmp.loc[frontier_mask].sort_values("rmia_mean")
        
    tst_list.append(tst)
    
print(tmp.shape, group.shape)

In [ ]:
# -------------------- #
#  Group Visualization #
# -------------------- #
plt.plot(tmp["rmia_mean"],tmp["accuracy_mean"],'.',label="RMIA Scatter")
plt.plot(tmp["lira_mean"],tmp["accuracy_mean"],'.',label="LiRA Scatter")
plt.plot(tst_list[0]["lira_mean"],tst_list[0]["accuracy_mean"],label="LiRA Pareto")
plt.plot(tst_list[1]["rmia_mean"],tst_list[1]["accuracy_mean"],label="RMIA Pareto")
plt.grid(True, alpha=0.3)
plt.legend(loc="lower right")
plt.xlabel("Membership Inference Vulnerability (τ)")
plt.ylabel("Accuracy")
plt.title(f"Pareto Frontiers of Accuracy vs. MIA Risk (FPR={grp_fpr})")
fig = plt.gcf()
fig.show()

savePlot(fig, f"pareto_scatter_{grp_fpr}", study_name, "study")